# **HelpBot**

A multi-agent for Customer Support Automation

- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

### **Install Required Packages**

In [1]:
%%capture --no-stderr
%pip install --quiet -U crewai
%pip install --quiet -U crewai_tools

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### **Import Required Libraries**

In [3]:
from crewai import Agent, Task, Crew, LLM
from google.colab import userdata
import os
from IPython.display import Markdown
from IPython.display import  display
import textwrap

### **Set Up API Key**

In [5]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

### **Initialize an LLM**

In [6]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.5,
)

### **Create Agents**

Define Agents, and provide them a `role`, `goal` and `backstory`.

In [7]:
# Agent: Support Agent

support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful "
         "support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        " are now working on providing "
        "support to {customer}, a super important customer "
        " for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [8]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the "
         "best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        "are now working with your team "
        "on a request from {customer} ensuring that "
        "the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative "
        "is providing full"
        "complete answers, and make no assumptions."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm,
)

### **Define Tools**

In [9]:
from crewai_tools import ScrapeWebsiteTool

In [10]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/quickstart"
)

### **Create Tasks**

In [11]:
# Task: Inquiry Resolution

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	      "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		    "Make sure to use everything you know "
        "to provide the best support possible."
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

In [12]:
# Task: Inquiry Assurance

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
        "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company "
        "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

### **Creating the Crew**

In [13]:
crew = Crew(
    agents=[support_agent,support_quality_assurance_agent],
    tasks=[inquiry_resolution,quality_assurance_review],
    verbose=True,
    memory=True,
    embedder={
        "provider": "google",
        "config": {
            "api_key": GEMINI_API_KEY,
             "model":"models/text-embedding-004"
        }
    }
)

### **Running the Crew and Displaying Response**

In [14]:
def to_markdown(text):
    # Replace bullet points with asterisks for markdown formatting
    text = text.replace("•", "  *")

    # Indent the text with a '>' character for blockquote style
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [15]:
inputs = {
    "customer": "PIAIC",
    "person": "Hassan Ali Khan",
    "inquiry": "I need help with setting up a crewAI agent"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: PIAIC just reached out with a super important ask:
I need help with setting up a crewAI agent

Hassan Ali Khan from PIAIC is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Thought: I need to find information on setting up a crewAI agent.  The provided tool allows me to access the quickstart guide on the crewAI website.  I will use that to get the necessary information.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Quickstart - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedQuickstartGet StartedExamplesCrewAI home pageCommunityChangelogGet StartedIntroductionInstallationQuickstartCore ConceptsAgentsTasksCrewsFlowsKnowledgeLLMsProcessesCollaborationTrainingMemoryPlannin

In [16]:
to_markdown(result.raw)

> Subject: Setting up your crewAI Agent - Let's get you started!
> 
> Hi Hassan Ali Khan,
> 
> Thanks for reaching out to crewAI support! We're thrilled to help you get started with your agent setup.  I understand you're looking to set up a crewAI agent, and I'm happy to guide you through the process.  I've taken a look at the quickstart guide (https://docs.crewai.com/quickstart) and crafted a more detailed and user-friendly explanation below.
> 
> Let's break down the setup into manageable steps.  Remember, if you get stuck at any point, please don't hesitate to ask! We're here to help.
> 
> **1. Project Creation and Initial Setup:**
> 
> First, let's create your crewAI project.  We'll use the command line for this.  The exact command might vary slightly depending on your operating system, but it generally follows this pattern:
> 
> ```bash
> crewai create my-piaic-agent
> ```
> 
> Replace `my-piaic-agent` with a descriptive name for your project. This command will generate a project directory with the necessary files: `agents.yaml`, `tasks.yaml`, `crew.py`, and potentially others.
> 
> **2. Defining Your Agents (`agents.yaml`):**
> 
> The `agents.yaml` file is where you define the agents that will participate in your crew.  Each agent is described with its:
> 
> * **`role`:**  What is this agent's function within the crew? (e.g., "Researcher," "Writer," "Summarizer")
> * **`goal`:** What is this agent trying to achieve? (e.g., "Find relevant research articles," "Write a concise summary," "Generate a creative story")
> * **`backstory`:** (Optional)  Providing a backstory can help shape the agent's behavior and responses.
> 
> Here's an example:
> 
> ```yaml
> agents:
>   - role: Researcher
>     goal: Find relevant research articles on AI ethics.
>     backstory: A seasoned researcher with a focus on the ethical implications of AI.
> ```
> 
> Remember, you can use variables within curly braces `{}` to dynamically populate values from your Python code (explained later).
> 
> **3. Defining Your Tasks (`tasks.yaml`):**
> 
> The `tasks.yaml` file outlines the tasks your agents will perform. Each task includes:
> 
> * **`description`:** A clear description of the task.
> * **`agent`:** The name of the agent responsible for this task (must match the name in `agents.yaml`).
> * **`output_file`:** (Optional) Specify a file where the task's output should be saved.  This is extremely helpful for organizing your results.
> 
> Example:
> 
> ```yaml
> tasks:
>   - description: "Research AI ethics articles"
>     agent: Researcher
>     output_file: research_results.txt
> ```
> 
> **4. Implementing Your Crew Logic (`crew.py`):**
> 
> This is the heart of your project.  Here, you'll write the Python code that defines the agents' behavior and how they interact.  We use decorators to link your Python code with your YAML configurations.
> 
> * `@CrewBase`:  This decorator marks the main class that orchestrates the crew.
> * `@agent`: This decorator defines an agent's functionality.
> * `@task`: This decorator defines a specific task within an agent.
> * `@crew`: This decorator defines the overall crew structure.
> 
> Here's a basic example using the `SerperDevTool`:
> 
> 
> ```python
> from crewai import CrewBase, agent, task, crew
> from serper import SerperAPI
> 
> @crew
> class MyCrew(CrewBase):
>     def __init__(self, api_key):
>         self.serper = SerperAPI(api_key)
> 
> @agent
> class Researcher(MyCrew):
>     @task
>     def research_ai_ethics(self):
>         results = self.serper.search("AI ethics")
>         with open("research_results.txt", "w") as f:
>             f.write(results)
> 
> # In your main script:
> my_crew = MyCrew(api_key="YOUR_SERPER_API_KEY")
> my_crew.run()
> 
> ```
> 
> Remember to replace `"YOUR_SERPER_API_KEY"` with your actual API key.
> 
> **5. Before and After Crew Functions (Optional):**
> 
> You can add functions that run before and after your crew executes using the `@before_kickoff` and `@after_kickoff` decorators.  This is useful for tasks like setting up the environment or cleaning up afterward.
> 
> **6. Custom Inputs:**
> 
> You can customize your crew's execution by passing a dictionary of variables to your main script.  These variables can then be accessed within your `crew.py` file.
> 
> **7. Environment Variables (.env file):**
> 
> Store sensitive information like your OpenAI API key and other keys in a `.env` file.  This keeps your keys out of your version control system.  You can use libraries like `python-dotenv` to load these variables into your script.
> 
> **8. Dependency Management:**
> 
> Use the crewAI CLI to lock and install your project's dependencies:
> 
> ```bash
> crewai lock
> crewai install
> ```
> 
> **9. Running Your Crew:**
> 
> Once everything is set up, run your crew using the following command:
> 
> ```bash
> crewai run
> ```
> 
> **10. Reviewing Your Results:**
> 
> The output of your crew will be displayed in the console, and any output files specified in your `tasks.yaml` will be created.
> 
> **Important Considerations:**
> 
> * **Naming Consistency:**  Maintain consistency between names in your YAML files and your Python code.
> * **Annotations:** Use the annotations `@agent`, `@task`, `@crew`, etc., correctly.
> * **Process Types:** crewAI supports both sequential and hierarchical processes.
> * **Replaying Tasks:** Use `crewai replay <task_id>` to rerun specific tasks.
> * **Resetting Crew Memory:** Use `crewai reset-memory` to clear the crew's memory.
> * **Deployment:** For easy deployment, consider CrewAI Enterprise.
> 
> 
> Please let me know if you have any questions or encounter any issues during the setup.  We're here to support you every step of the way!  We value your business and are committed to your success with crewAI.
> 
> Best regards,
> 
> [Your Name]
> Senior Support Representative
> crewAI

---